# Data importing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
activities = pd.read_csv('../data/activities.csv')
activities

In [ ]:
# 600 events for each contributor
data = ( 
    activities
    .groupby('contributor')
    .tail(600)
    .groupby('contributor')
    .filter(lambda x: len(x) == 600)
)
data['date'] = pd.to_datetime(data['date'])
data

# Dynamic frequence

In [ ]:
def map_frequency(time_range):
    if time_range >= 60 * 60 * 24:
        return 'D'  # Daily if time range for one activity is a day or more
    elif time_range >= 60 * 60:
        time_range_h = time_range // 3600
        freq = int(24 // ((24 // time_range_h) + 1)) if time_range_h > 1 else 1
        return f"{freq}H"  # Hourly frequency
    elif time_range >= 60:
        time_range_m = time_range // 60
        freq = int(60 // ((60 // time_range_m) + 1)) if time_range_m > 1 else 1
        return f"{freq}T"  # Minutely frequency
    else:
        freq = int(60 // ((60 // time_range) + 1)) if time_range > 1 else 1
        return f"{freq}S"  # Secondly frequency

In [ ]:
def frequency_to_seasonality(freq):
    unit = freq[-1]
    if unit == 'D':
        return 7
    elif unit == 'H':
        return 24 // int(freq[:-1])
    elif unit == 'T':
        return 24 * (60 // int(freq[:-1]))
    else:
        return None

In [ ]:
# ((24*60*60)//((24*60*60)//(49*60*60)+1))//3600

In [ ]:
random_name = data['contributor'].sample().values[0]
random_user = data[data['contributor'] == random_name].reset_index(drop=True)

result = (
    random_user
    .groupby([pd.Grouper(key='date', freq='D')])['activity']
    .count()
    .reset_index(name='n_activities')
    .set_index('date')
    .resample('D')
    .sum()
    .rename_axis(None)
)
result.plot(figsize=(14, 6))

## 1. Mean approach

Find the frequency based on the mean of time differences between activities

In [ ]:
def mean_based_frequency(contributor):
    train_data = contributor.head(300)
    train_data = train_data[train_data['date'] >= train_data['date'].max() - pd.DateOffset(months=3)]
    time_range = (train_data['date'].iloc[-1] - train_data['date'].iloc[0]).total_seconds()//len(train_data)
    print("Time range is :", time_range)
    frequency = map_frequency(time_range)
    return frequency

In [ ]:
# Example usage:
print("Frequence is :", mean_based_frequency(random_user))

## 2. Quantile approach

Find the frequency based on the median of time differences between activities

In [ ]:
def quantile_based_frequency(contributor, quantile=0.5):
    train_data = contributor.head(300)
    train_data = train_data[train_data['date'] >= train_data['date'].max() - pd.DateOffset(months=3)]
    train_data['time_diff'] = train_data['date'].diff().dt.total_seconds()
    time_range = train_data['time_diff'].quantile(quantile)
    print("Time range is :", time_range)
    frequency = map_frequency(time_range)
    return frequency

In [ ]:
# Example usage:
print("Frequence is :", quantile_based_frequency(random_user, quantile=0.5))

## 3. Smallest time approach

Find the frequency based on the smallest time to do a specified number of activities (= 100) by sliding over the data

In [ ]:
def smallest_time_based_frequency(contributor, window_size=100):
    train_data = contributor.head(300)
    train_data = train_data[train_data['date'] >= train_data['date'].max() - pd.DateOffset(months=3)]

    smallest_time = None
    # iterate over contributor's data in a sliding window
    for i in range(len(train_data) - window_size + 1):
        window = train_data.iloc[i:i + window_size]
        time_span = window['date'].iloc[-1] - window['date'].iloc[0]

        # the smallest time span
        if smallest_time is None or time_span < smallest_time:
            smallest_time = time_span

    time_range = smallest_time.total_seconds()//window_size

    print("Time range is :", time_range)
    frequency = map_frequency(time_range)

    return frequency

In [ ]:
# Example usage:
print("Frequence is :", smallest_time_based_frequency(random_user, window_size=100))

# Data splitting

In [ ]:
def split_activities(contributor, frequency):

    result = (
        contributor
        .groupby([pd.Grouper(key='date', freq=frequency)])['activity']
        .count()
        .reset_index(name='n_activities')
        .set_index('date')
        .resample(frequency)
        .sum()
        .rename_axis(None)
        .assign(
            cumsum_activities = lambda x: x['n_activities'].cumsum()
        )
    )

    train, test = (
        result[result['cumsum_activities'] <= 300]
        .apply(lambda x : x[x.index >= x.index.max() - pd.DateOffset(months=3)])
        .drop('cumsum_activities', axis=1),

        result[result['cumsum_activities'] > 300]
        .drop('cumsum_activities', axis=1)
    )

    return train, test

In [ ]:
frequency = smallest_time_based_frequency(random_user, window_size=100)
train, test = split_activities(random_user, frequency)
train.plot()

# Evaluation metrics

In [ ]:
def ctd_score(y_true, y_pred, target_value):
    true_cumsum = np.cumsum(y_true)
    pred_cumsum = np.cumsum(y_pred)

    if np.sum(y_pred) < target_value or np.sum(y_true) < target_value:
        return None

    time_true = np.argmax(true_cumsum >= target_value)
    time_pred = np.argmax(pred_cumsum >= target_value)

    return time_true - time_pred

# SARIMA model

In [ ]:
def sarima_model(contributor):

    contributor_name = contributor['contributor'].iloc[0]
    contributor_category = contributor['category'].iloc[0]

    frequency = mean_based_frequency(contributor)
    seasonality = frequency_to_seasonality(frequency)

    train, test = split_activities(contributor, frequency)

    print(contributor_name, frequency, seasonality)

    # choosing best parameters (possible seasonality)
    if seasonality and (len(train)//seasonality) >= 2:
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, seasonality)
    else:
        order = (int((len(train)/3)), 1, 0)
        seasonal_order = None
        seasonality = None

    model = SARIMAX(
        train['n_activities'],
        order = order,
        seasonal_order = seasonal_order, 
        enforce_invertibility = True, 
        enforce_stationarity = False
        ).fit(disp=False, method='lbfgs')

    # Forecast the test set using confidence interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    metrics = pd.Series({
        'contributor': contributor_name,
        'category': contributor_category,
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'frequency': frequency,
        'seasonality':seasonality,
        'data_points':len(train),
    })

    return metrics

In [ ]:
sarima_results = data.groupby(['category', 'contributor']).apply(sarima_model).reset_index(drop=True)

In [ ]:
sarima_results.to_csv('../models-evaluation/sarima_model-mean-v1.csv', index=False)

# Unobserved components model

In [ ]:
def uc_model(contributor):

    contributor_name = contributor['contributor'].iloc[0]
    contributor_category = contributor['category'].iloc[0]

    frequency = mean_based_frequency(contributor)
    seasonality = frequency_to_seasonality(frequency)

    train, test = split_activities(contributor, frequency)

    print(contributor_name, frequency, seasonality)

    # choosing best parameters (possible seasonality)
    if (len(train)//seasonality) < 2:
        seasonality = None

    model = UnobservedComponents(
        train['n_activities'], 
        level = True, 
        seasonal = seasonality
        ).fit(disp=False, method='lbfgs')
    
    # Forecast the test set using confidence interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    metrics = pd.Series({
        'contributor': contributor_name,
        'category': contributor_category,
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'frequency': frequency,
        'seasonality':seasonality,
        'data_points':len(train),
    })

    return metrics

In [ ]:
uc_results = data.groupby(['category', 'contributor']).apply(uc_model).reset_index(drop=True)

In [ ]:
uc_results.to_csv('../models-evaluation/uc_model-mean-v1.csv', index=False)

# Error Trend and Seasonality model

In [ ]:
def ets_model(contributor):

    contributor_name = contributor['contributor'].iloc[0]
    contributor_category = contributor['category'].iloc[0]

    frequency = mean_based_frequency(contributor)
    seasonality = frequency_to_seasonality(frequency)

    train, test = split_activities(contributor, frequency)

    print(contributor_name, frequency, seasonality)

    # choosing best parameters (possible seasonality)
    if seasonality and (len(train)//seasonality) >= 2:
        seasonal = 'add'
    else:
        seasonality = None
        seasonal = None

    model = ETSModel(
        train['n_activities'], 
        error = 'add', 
        trend = 'add', 
        seasonal = seasonal, 
        seasonal_periods = seasonality
    ).fit(disp=False)

    # Forecast the test set using confidence interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    metrics = pd.Series({
        'contributor': contributor_name,
        'category': contributor_category,
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'frequency': frequency,
        'seasonality':seasonality,
        'data_points':len(train),
    })

    return metrics

In [ ]:
ets_results = data.groupby(['category', 'contributor']).apply(ets_model).reset_index(drop=True)

In [ ]:
ets_results.to_csv('../models-evaluation/ets_model-mean-v1.csv', index=False)